<a href="https://colab.research.google.com/github/ahmedse/mlops/blob/tfx/simple_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [2]:
!pip install -U tfx

     |████████████████████████████████| 2.4 MB 5.0 MB/s            
     |████████████████████████████████| 1.4 MB 41.6 MB/s            
     |████████████████████████████████| 98 kB 6.9 MB/s             
     |████████████████████████████████| 1.7 MB 27.7 MB/s            
     |████████████████████████████████| 422 kB 74.3 MB/s            
     |████████████████████████████████| 1.7 MB 63.0 MB/s            
     |████████████████████████████████| 135 kB 63.2 MB/s            
     |████████████████████████████████| 9.9 MB 51.8 MB/s            
     |████████████████████████████████| 6.6 MB 52.4 MB/s            
     |████████████████████████████████| 19.1 MB 373 kB/s             
     |████████████████████████████████| 1.8 MB 53.7 MB/s            
     |████████████████████████████████| 1.2 MB 36.2 MB/s            
     |████████████████████████████████| 40 kB 5.2 MB/s             
     |████████████████████████████████| 147 kB 62.7 MB/s            
     |██████████████████████████████